# 概要
これは落書き用のノートブックです

# 使い方
- 試したいことをこのノートブックに書く
- 残しておきたいことはこのブックからコピーして別ファイルを作成する

# GoogleMapsのAPI使ってみた
- 場所の詳細情報をゲットするには`gmap.palces`で`place_id`を手に入れてから、`gmap.place`に`place_id`を渡す

In [3]:
!pip install --use-pep517 googlemaps

  Using cached googlemaps-4.10.0.tar.gz (33 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40713 sha256=248f5a17e30e1c6e73415d62188317796db8ef644843da808c28c1572ec811c2
  Stored in directory: /Users/OgawaAyumu/Library/Caches/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [40]:
import googlemaps
import dotenv
import os
import subprocess

In [2]:
dotenv.load_dotenv()

True

In [3]:
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
if not GOOGLE_MAPS_API_KEY:
    raise Exception("GOOGLE_MAPS_API_KEY not found")

In [4]:
gmap = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

In [21]:
res = gmap.places("慈照寺")

In [22]:
res["results"][0]["place_id"]

'ChIJ4W9CCwUJAWARyauI6BzKiiU'

In [23]:
res1 = gmap.place(res["results"][0]["place_id"])

In [24]:
res1["result"]

{'address_components': [{'long_name': '2',
   'short_name': '2',
   'types': ['premise']},
  {'long_name': 'Ginkakujichō',
   'short_name': 'Ginkakujichō',
   'types': ['sublocality_level_2', 'sublocality', 'political']},
  {'long_name': 'Sakyo Ward',
   'short_name': 'Sakyo Ward',
   'types': ['sublocality_level_1', 'sublocality', 'political']},
  {'long_name': 'Kyoto',
   'short_name': 'Kyoto',
   'types': ['locality', 'political']},
  {'long_name': 'Kyoto',
   'short_name': 'Kyoto',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Japan',
   'short_name': 'JP',
   'types': ['country', 'political']},
  {'long_name': '606-8402',
   'short_name': '606-8402',
   'types': ['postal_code']}],
 'adr_address': '<span class="street-address">2 Ginkakujichō, Sakyo Ward, Kyoto</span>, <span class="postal-code">606-8402</span>, <span class="country-name">Japan</span>',
 'business_status': 'OPERATIONAL',
 'current_opening_hours': {'open_now': True,
  'periods': [{'close'

In [81]:
spots = ["京都大学吉田キャンパス", "吉田神社", "銀閣寺", "哲学の道", "南禅寺", "岡崎神社", "平安神宮", "永観堂", "下鴨神社", "京都御苑"]

In [82]:
d_matrix = gmap.distance_matrix(origins=spots,destinations=spots)

In [83]:
dist = [[row["elements"][i]["duration"]["value"] for i in range(len(d_matrix["rows"]))] for row in d_matrix["rows"]]

In [84]:
with open("dist.txt", "w") as f:
    f.write(f"{len(dist)}\n"+"\n".join([" ".join([str(d) for d in di]) for di in dist]))

In [85]:
res = subprocess.run("./dist/bitdp < dist.txt", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [86]:
lines = res.stdout.decode("utf-8").split("\n")
sec = int(lines[0])
route = list(map(int, lines[1].split()))

In [88]:
print(f"{sec//60}m {sec%60}s")
print("route:")
for i in route:
    print(spots[i])

51m 4s
route:
京都大学吉田キャンパス
平安神宮
岡崎神社
南禅寺
永観堂
哲学の道
銀閣寺
吉田神社
京都御苑
下鴨神社
京都大学吉田キャンパス
